In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
dataset = pd.read_csv('processed_cleveland.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
dataset.fillna(dataset['thal'].median(), inplace = True)
dataset.fillna(dataset['ca'].median(), inplace = True)

In [ ]:
dataset.describe()
Q1 = dataset.num.quantile(0.35);
Q3 = dataset.num.quantile(0.65);
Q1, Q3
IQR = Q3 - Q1
IQR
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
lower_limit, upper_limit
dataset[(dataset.num<lower_limit)|(dataset.num>upper_limit)]
df_no_outlier = dataset[(dataset.num>lower_limit)&(dataset.num<upper_limit)]
print(df_no_outlier)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   1       145   233    1        2      150      0      2.3   
1     67    1   4       160   286    0        2      108      1      1.5   
2     67    1   4       120   229    0        2      129      1      2.6   
3     37    1   3       130   250    0        0      187      0      3.5   
4     41    0   2       130   204    0        2      172      0      1.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
297   57    0   4       140   241    0        0      123      1      0.2   
298   45    1   1       110   264    0        0      132      0      1.2   
299   68    1   4       144   193    1        0      141      0      3.4   
301   57    0   2       130   236    0        2      174      0      0.0   
302   38    1   3       138   175    0        0      173      0      0.0   

     slope   ca  thal  num  
0        3  0.0   6.0    0  
1        2  3.0   3.0    2  


In [ ]:
df_no_outlier.describe()
Q1 = df_no_outlier.num.quantile(0.25)
Q3 = df_no_outlier.num.quantile(0.75)
Q1, Q3
IQR = Q3 - Q1
IQR
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 0.5*IQR
lower_limit, upper_limit
df_no_outlier1 = dataset[(dataset.num>lower_limit)&(dataset.num<upper_limit)]
print(df_no_outlier1)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   1       145   233    1        2      150      0      2.3   
2     67    1   4       120   229    0        2      129      1      2.6   
3     37    1   3       130   250    0        0      187      0      3.5   
4     41    0   2       130   204    0        2      172      0      1.4   
5     56    1   2       120   236    0        0      178      0      0.8   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
295   41    1   2       120   157    0        0      182      0      0.0   
297   57    0   4       140   241    0        0      123      1      0.2   
298   45    1   1       110   264    0        0      132      0      1.2   
301   57    0   2       130   236    0        2      174      0      0.0   
302   38    1   3       138   175    0        0      173      0      0.0   

     slope   ca  thal  num  
0        3  0.0   6.0    0  
2        2  2.0   7.0    1  


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from zoofs import HarrisHawkOptimization

# Define your own objective function for HHO
def objective_function(model, X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    return accuracy

# Create an instance of randomforest
dt_model = RandomForestClassifier(n_estimators=100, random_state=0)

# Create an object of HarrisHawkOptimization
algo_object = HarrisHawkOptimization(
    objective_function,
    n_iteration=10,         # Number of iterations
    population_size=50,     # Population size
    minimize=False          # Set to True if your objective is to minimize
)

# Fit the algorithm
algo_object.fit(dt_model, X_train, y_train, X_valid, y_valid, verbose=True)

# Get the best feature list
max_features = 5 # Set the maximum number of features you want
best_feature_list = algo_object.best_feature_list
limited_feature_list = best_feature_list[:max_features]
print(limited_feature_list)
# Train the Decision Tree model using the best selected features
dt_model.fit(X_train.loc[:, best_feature_list], y_train)

# Predict using the test dataset
y_pred = dt_model.predict(X_valid.loc[:,best_feature_list ])

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy:", accuracy)
accuracy_percent=accuracy*100
print("accuracy in percentage",accuracy_percent,"%")

 [ 2023-10-27 22:08:46,775 ] Finished iteration #0 with objective value 0.8787878787878788. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:08:54,522 ] Finished iteration #1 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:03,020 ] Finished iteration #2 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:11,607 ] Finished iteration #3 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:20,882 ] Finished iteration #4 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:29,052 ] Finished iteration #5 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:36,227 ] Finished iteration #6 with objective value 0.9090909090909091. Current best value is 0.9090909090909091 
 [ 2023-10-27 22:09:44,362 ] Finished iteration #7 with object

['age', 'sex', 'cp', 'trestbps', 'chol']
Accuracy: 0.9242424242424242
accuracy in percentage 92.42424242424242 %
